In [28]:
from keras.applications import vgg16
model = vgg16.VGG16(weights=None, input_shape=[201, 201, 2], classes=17)

/usr/lib64/python3.6/site-packages/keras_applications/imagenet_utils.py:279: UserWarning: This model usually expects 1 or 3 input channels. However, it was passed an input_shape with 2 input channels.
  str(input_shape[-1]) + ' input channels.')


In [29]:
# https://medium.com/@thongonary/how-to-compute-f1-score-for-each-epoch-in-keras-a1acd17715a2
import numpy as np
from keras.callbacks import Callback
from sklearn.metrics import confusion_matrix, f1_score, precision_score, recall_score

class Metrics(Callback):
    def on_train_begin(self, logs={}):
        self.val_f1s = []
        self.val_recalls = []
        self.val_precisions = []
    
    def on_epoch_end(self, epoch, logs={}):
        # https://stackoverflow.com/a/51517498
        val_predict = (np.asarray(self.model.predict(self.validation_data[0]))).round()
        val_targ = self.model.validation_data[1]
        _val_f1 = f1_score(val_targ, val_predict)
        _val_recall = recall_score(val_targ, val_predict)
        _val_precision = precision_score(val_targ, val_predict)
        self.val_f1s.append(_val_f1)
        self.val_recalls.append(_val_recall)
        self.val_precisions.append(_val_precision)
        print(" — val_f1: %f — val_precision: %f — val_recall %f" % (_val_f1, _val_precision, _val_recall))
        return
 
metrics = Metrics()

In [30]:
model.compile(loss='categorical_crossentropy', optimizer='adam')

In [31]:
from dataGenerator import DataGenerator
from personal_data import location_data
dataGenerator_train = DataGenerator(location_data, batch_size=128)
dataGenerator_val = DataGenerator(location_data, batch_size=128, isTrainingData=False)

In [32]:
from keras import callbacks
tensorboard = callbacks.TensorBoard(log_dir='tensorboard', update_freq='batch')

SyntaxError: positional argument follows keyword argument (<ipython-input-32-1765cb742cb1>, line 2)

In [0]:
model.fit_generator(dataGenerator_train.generate(),
    validation_data=dataGenerator_val.generate(),
    callbacks = [metrics, tensorboard],
    steps_per_epoch=len(dataGenerator_train),
    validation_steps=len(dataGenerator_val),
    verbose=1, epochs=100)